# Cincinnati Model Architecture Part 1

In this notebook, the model architecture from the NYC taxi data Part 7 is applied to the Cincinnati Vehicle DataSet.
This is done to verify whether the network behaves the same for both datasets.


In [1]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


# Dataset

In [3]:
df = pd.read_csv("./cincinnati/cincinatti_zones_153.csv")
df.head(10)

,location_id,vehicle_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,4,153,False,-0.725424,-0.688302,0.743145,0.669131,0.5,0.866025,0.0,1.0
1,10,153,False,-0.619379,-0.785092,0.743145,0.669131,0.5,0.866025,0.0,1.0
2,4,153,False,0.378379,-0.925651,0.743145,0.669131,0.5,0.866025,0.0,1.0
3,4,153,False,0.635798,-0.771856,0.743145,0.669131,0.5,0.866025,0.0,1.0
4,11,153,False,-0.492424,-0.870356,0.743145,0.669131,0.5,0.866025,0.0,1.0
5,4,153,False,0.476903,-0.878956,0.743145,0.669131,0.5,0.866025,0.0,1.0
6,18,153,False,-0.697999,-0.716099,0.743145,0.669131,0.5,0.866025,0.0,1.0
7,10,153,False,0.121725,-0.992564,0.743145,0.669131,0.5,0.866025,0.0,1.0
8,10,153,False,0.004363,-0.999990,0.743145,0.669131,0.5,0.866025,0.0,1.0
9,13,153,False,0.618923,-0.785452,0.743145,0.669131,0.5,0.866025,0.0,1.0


In [4]:
df.drop(columns=['vehicle_id'], axis='columns', inplace=True)
df

,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,4,False,-0.725424,-0.688302,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
1,10,False,-0.619379,-0.785092,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
2,4,False,0.378379,-0.925651,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
3,4,False,0.635798,-0.771856,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
4,11,False,-0.492424,-0.870356,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
165636,0,False,-0.108939,-0.994048,-0.587785,0.809017,-0.500000,-0.866025,0.781831,0.623490
165637,0,False,-0.083026,-0.996547,0.743145,0.669131,-0.866025,-0.500000,0.974928,-0.222521
165638,1,True,0.321302,-0.946977,0.207912,0.978148,-0.500000,-0.866025,-0.974928,-0.222521
165639,1,False,-0.933554,0.358436,0.743145,0.669131,-0.866025,-0.500000,0.974928,-0.222521


In [5]:
# the number of different locations defines the vocabulary size
locations = df.location_id
vocab_size = locations.nunique()
print('vocabulary size:', vocab_size)

vocabulary size: 47


In [6]:
df['location_id'] = df.groupby('location_id', sort=False).ngroup()
df.head(10)

,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,0,False,-0.725424,-0.688302,0.743145,0.669131,0.5,0.866025,0.0,1.0
1,1,False,-0.619379,-0.785092,0.743145,0.669131,0.5,0.866025,0.0,1.0
2,0,False,0.378379,-0.925651,0.743145,0.669131,0.5,0.866025,0.0,1.0
3,0,False,0.635798,-0.771856,0.743145,0.669131,0.5,0.866025,0.0,1.0
4,2,False,-0.492424,-0.870356,0.743145,0.669131,0.5,0.866025,0.0,1.0
5,0,False,0.476903,-0.878956,0.743145,0.669131,0.5,0.866025,0.0,1.0
6,3,False,-0.697999,-0.716099,0.743145,0.669131,0.5,0.866025,0.0,1.0
7,1,False,0.121725,-0.992564,0.743145,0.669131,0.5,0.866025,0.0,1.0
8,1,False,0.004363,-0.999990,0.743145,0.669131,0.5,0.866025,0.0,1.0
9,4,False,0.618923,-0.785452,0.743145,0.669131,0.5,0.866025,0.0,1.0


In [7]:
mh = ModelHelper(df, 17)

In [15]:
mh.set_target_column_name('location_id')
mh.set_vocab_size(vocab_size)
numerical_column_names = ['start_hour_sin', 'start_hour_cos', 'weekend', 'week_day_sin', 'week_day_cos']
column_names = ['location_id'] + numerical_column_names
mh.set_column_names(column_names)
mh.set_numerical_column_names(numerical_column_names)

In [16]:
# Call the function
mh.train_val_test_split()
print(len(mh.df_train), 'train examples')
print(len(mh.df_val), 'validation examples')
print(len(mh.df_test), 'test examples')

106009 train examples
26503 validation examples
33129 test examples


In [17]:
mh.split_data()
print(len(mh.list_test))
mh.list_test[0]

1949


,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
132512,34,False,-0.915429,-0.402480,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132513,8,False,-0.999767,-0.021597,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132514,15,False,0.186524,-0.982450,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132515,4,False,-0.694815,-0.719188,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132516,4,False,-0.865989,0.500063,0.994522,-0.104528,-0.5,-0.866025,0.433884,-0.900969
132517,12,False,0.216298,-0.976327,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132518,10,False,-0.026468,-0.999650,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132519,18,False,-0.978986,0.203927,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132520,8,False,-0.918734,0.394877,0.994522,-0.104528,-0.5,-0.866025,0.433884,-0.900969
132521,8,False,-0.995792,0.091646,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969


In [18]:
mh.set_batch_size(16)
mh.create_batch_dataset_cin()
mh.test_dataset

<BatchDataset shapes: ({start_place: (16, 16), start_hour_sin: (16, 16), start_hour_cos: (16, 16), weekend: (16, 16), week_day_sin: (16, 16), week_day_cos: (16, 16), end_hour_sin: (16,), end_hour_cos: (16,), end_weekend: (16,), end_week_day_sin: (16,), end_week_day_cos: (16,)}, (16,)), types: ({start_place: tf.int64, start_hour_sin: tf.float64, start_hour_cos: tf.float64, weekend: tf.bool, week_day_sin: tf.float64, week_day_cos: tf.float64, end_hour_sin: tf.float64, end_hour_cos: tf.float64, end_weekend: tf.bool, end_week_day_sin: tf.float64, end_week_day_cos: tf.float64}, tf.int64)>

In [19]:
def sparse_f(input_dense):
  zero = tf.constant(0, dtype=tf.float32)
  indices = tf.where(tf.not_equal(input_dense, zero))
  values = tf.gather_nd(input_dense, indices)
  sparse = tf.SparseTensor(indices, values,  tf.cast(tf.shape(input_dense), dtype=tf.int64))
  return sparse

In [28]:
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units1 = 256
rnn_units2 = 128

# Create a model
def create_model():
  N = mh.total_window_length
  batch_size = mh.batch_size
  number_of_places = mh.vocab_size

  # Shortcut to the layers package
  l = tf.keras.layers

  other_feature_inputs = {
    'end_hour_sin': tf.keras.Input((1, ), batch_size=batch_size, name='end_hour_sin'),
    'end_hour_cos': tf.keras.Input((1, ), batch_size=batch_size, name='end_hour_cos'),
    'end_weekend': tf.keras.Input((1, ), batch_size=batch_size, name='end_weekend'),
    'end_week_day_sin': tf.keras.Input((1, ), batch_size=batch_size, name='end_week_day_sin'),
    'end_week_day_cos': tf.keras.Input((1, ), batch_size=batch_size, name='end_week_day_cos')
  }

  # List of numeric feature columns to pass to the DenseLayer
  num_features = []
  feature_inputs={}
  # Handling numerical columns
  for header in mh.numerical_column_names:
    input = tf.keras.Input((N-1,), dtype=tf.dtypes.float32, batch_size=batch_size, name=header)
    feature_inputs[header] = input
    f =  feature_column.sequence_numeric_column(header, shape=(N-1), dtype=tf.dtypes.float32)
    sparse_input = tf.keras.layers.Lambda(sparse_f)(input)
    feature, sequence_length = tf.keras.experimental.SequenceFeatures(f)({header: sparse_input})
    feature = tf.reshape(feature, (sparse_input.shape[0], sparse_input.shape[1], 1))
    num_features.append(feature)

  end_hour_sin = feature_column.numeric_column("end_hour_sin", shape=(1))
  end_hour_sin_feature = l.DenseFeatures(end_hour_sin)(other_feature_inputs)

  end_hour_cos = feature_column.numeric_column("end_hour_cos", shape=(1))
  end_hour_cos_feature = l.DenseFeatures(end_hour_cos)(other_feature_inputs)

  end_weekend = feature_column.numeric_column("end_weekend", shape=(1))
  end_weekend_feature = l.DenseFeatures(end_weekend)(other_feature_inputs)

  end_week_day_sin = feature_column.numeric_column("end_week_day_sin", shape=(1))
  end_week_day_sin_feature = l.DenseFeatures(end_week_day_sin)(other_feature_inputs)

  end_week_day_cos = feature_column.numeric_column("end_week_day_cos", shape=(1))
  end_week_day_cos_feature = l.DenseFeatures(end_week_day_cos)(other_feature_inputs)


  # Declare the dictionary for the places sequence as before
  sequence_input = {
      'location_id': tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name='location_id') # add batch_size=batch_size in case of stateful GRU
  }


  # Handling the categorical feature sequence using one-hot
  places_one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(
      'location_id', [i for i in range(number_of_places)])

  # Embed the one-hot encoding
  places_embed = feature_column.embedding_column(places_one_hot, embedding_dim)


  # With an input sequence we can't use the DenseFeature layer, we need to use the SequenceFeatures
  sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(places_embed)(sequence_input)

  input_sequence = l.Concatenate(axis=2)([sequence_features] + num_features)

  # Rnn
  recurrent = l.GRU(rnn_units1,
                        batch_size=batch_size, #in case of stateful
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(input_sequence)

  recurrent_2 = l.GRU(rnn_units2,
                        batch_size=batch_size, #in case of stateful
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(recurrent)


  flatten = l.Flatten()(recurrent_2)

  concatenate_2 = l.Concatenate(axis=1)([flatten, end_hour_sin_feature, end_hour_cos_feature, end_weekend_feature, end_week_day_sin_feature, end_week_day_cos_feature])

  # Last layer with an output for each places
  dense_1 = layers.Dense(number_of_places)(concatenate_2)

	# Softmax output layer
  output = l.Softmax()(dense_1)

  # To return the Model, we need to define its inputs and outputs
  # In out case, we need to list all the input layers we have defined
  inputs = list(other_feature_inputs.values()) + list(feature_inputs.values()) + list(sequence_input.values())

  # Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [29]:
# Get the model and compile it
mh.assign_model(create_model())
mh.compile_model()

In [30]:
mh.model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
start_hour_sin (InputLayer)     [(16, 16)]           0                                            
__________________________________________________________________________________________________
start_hour_cos (InputLayer)     [(16, 16)]           0                                            
__________________________________________________________________________________________________
weekend (InputLayer)            [(16, 16)]           0                                            
__________________________________________________________________________________________________
week_day_sin (InputLayer)       [(16, 16)]           0                                            
_______________________________________________________________________________________

In [31]:
mh.set_num_epochs(10)
mh.fit_model()

Epoch 1/10
389/389 [==============================] - 6s 15ms/step - loss: 2.4377 - sparse_categorical_accuracy: 0.2331 - val_loss: 2.5542 - val_sparse_categorical_accuracy: 0.2500
Epoch 2/10
389/389 [==============================] - 5s 13ms/step - loss: 2.2492 - sparse_categorical_accuracy: 0.2600 - val_loss: 2.4350 - val_sparse_categorical_accuracy: 0.2610
Epoch 3/10
389/389 [==============================] - 5s 13ms/step - loss: 2.1576 - sparse_categorical_accuracy: 0.2820 - val_loss: 2.3762 - val_sparse_categorical_accuracy: 0.2545
Epoch 4/10
389/389 [==============================] - 6s 14ms/step - loss: 2.0893 - sparse_categorical_accuracy: 0.2932 - val_loss: 2.3615 - val_sparse_categorical_accuracy: 0.2584
Epoch 5/10
389/389 [==============================] - 5s 13ms/step - loss: 2.0356 - sparse_categorical_accuracy: 0.3043 - val_loss: 2.3262 - val_sparse_categorical_accuracy: 0.2642
Epoch 6/10
389/389 [==============================] - 5s 13ms/step - loss: 1.9849 - sparse_cate

# Evaluation

In [32]:
mh.evaluate_model()

121/121 [==============================] - 0s 4ms/step - loss: 2.3032 - sparse_categorical_accuracy: 0.2805


In [33]:
mh.model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
start_hour_sin (InputLayer)     [(16, 16)]           0                                            
__________________________________________________________________________________________________
start_hour_cos (InputLayer)     [(16, 16)]           0                                            
__________________________________________________________________________________________________
weekend (InputLayer)            [(16, 16)]           0                                            
__________________________________________________________________________________________________
week_day_sin (InputLayer)       [(16, 16)]           0                                            
_______________________________________________________________________________________

In [34]:
mh.print_test_prediction_info()

logits
Shape :  (1936, 47)
Example [0] :  [1.57704458e-01 3.10565275e-03 2.65862197e-02 2.65136659e-02
 1.94100678e-01 1.31298788e-03 2.31216941e-02 8.71297438e-03
 2.63458371e-01 3.56368460e-02 2.54922286e-02 7.74704665e-03
 2.17520706e-02 6.62570761e-04 1.05308294e-02 1.82004534e-02
 1.99298447e-04 1.31864613e-02 1.46502569e-01 3.01128626e-03
 2.01009517e-03 6.26325491e-04 4.22046151e-06 3.00164946e-04
 1.17898197e-03 1.04657176e-03 3.63969375e-05 8.25707139e-06
 4.53654589e-04 5.39265864e-04 2.09326899e-04 4.48016726e-05
 1.62637971e-05 1.52031571e-04 2.16269041e-06 6.15412660e-04
 6.15839162e-05 7.63357602e-05 1.95857792e-04 4.70457817e-05
 2.81296950e-03 1.08262375e-05 4.35253663e-04 1.55354117e-03
 5.89341380e-06 3.53395649e-06 1.50701135e-05]
predictions
Shape :  (1936, 47)
Example [0] :  tf.Tensor(
[0.02434894 0.02086117 0.0213568  0.02135525 0.02525147 0.02082381
 0.02128294 0.02097847 0.02706502 0.02155097 0.02133345 0.02095822
 0.02125381 0.02081027 0.02101664 0.02117845 0.0

Next, we have to compare it to the reduced model and see if this performs better.